In [1]:
import pandas as pd

contem_2023 = pd.read_csv("/shared/share_tm-finance/Processed_df/One_year_window/contem_2023.csv")

In [2]:
contem_2023.head()

,date,rp_entity_id,comnam,ret,headline,vocab_con_headline
0,2023-01-03,3DC887,ARISTA NETWORKS INC,-0.003626,Network Monitoring Global Market Report 2022: ...,network monitoring sector reach 3.8 2030 cagr
1,2023-01-03,3DC887,ARISTA NETWORKS INC,-0.003626,Network Monitoring Global Market Report 2022: ...,network monitoring corporations continue onlin...
2,2023-01-03,3DC887,ARISTA NETWORKS INC,-0.003626,iRocket Appoints Kelyn Brannon to Board of Dir...,irocket appoints kelyn brannon board directors
3,2023-01-03,8EA478,MODERNA INC,-0.003507,Official List Official List Notice -2-,official list official list notice
4,2023-01-03,8EA478,MODERNA INC,-0.003507,DelveInsight Evaluates a Robust Cystic Fibrosi...,delveinsight evaluates robust cystic fibrosis ...


In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm

headlines = contem_2023.vocab_con_headline.to_list()
vectorizer = CountVectorizer()
doc_term = vectorizer.fit_transform(headlines)

In [4]:
from lda import LDA
ldamodel = LDA(n_topics = 62, n_iter = 100, random_state = 1)
ldamodel.fit(doc_term)

INFO:lda:n_documents: 1749222
INFO:lda:vocab_size: 115298
INFO:lda:n_words: 12456812
INFO:lda:n_topics: 62
INFO:lda:n_iter: 100
INFO:lda:<0> log likelihood: -174535983
INFO:lda:<10> log likelihood: -135300727
INFO:lda:<20> log likelihood: -117777713
INFO:lda:<30> log likelihood: -114191000
INFO:lda:<40> log likelihood: -112673734
INFO:lda:<50> log likelihood: -111859048
INFO:lda:<60> log likelihood: -111329404
INFO:lda:<70> log likelihood: -110980483
INFO:lda:<80> log likelihood: -110707711
INFO:lda:<90> log likelihood: -110504943
INFO:lda:<99> log likelihood: -110331493


In [5]:
import numpy as np

topic_dist = ldamodel.doc_topic_

contem_2023_topic_dist = pd.concat([contem_2023.drop(columns = ["rp_entity_id","headline","vocab_con_headline"]),\
                                    pd.DataFrame(topic_dist)], axis =1)
grouped_sum = contem_2023_topic_dist.groupby(["date","comnam","ret"]).sum()

X = np.array(grouped_sum)
ret = [ind[2] for ind in list(grouped_sum.index)]
Y = np.array(ret).reshape(-1,1)

from sklearn.linear_model import LinearRegression
lda_model = LinearRegression(fit_intercept=True)
lda_model.fit(X,Y)
lda_model.score(X,Y)

0.009622704975147833

In [12]:
# import pickle

# with open("/user/wx2309/Topic-modeling-store/LDA_sp500_2023_contem_model_folder/LDA_sp500_2023_contem_model.pkl", "wb") as file:
#     pickle.dump(ldamodel,file)